In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import os

c:\Users\rahul\.conda\envs\rahul\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
df_lyrics = pd.read_csv(r"C:\Users\rahul\Downloads\25klyricsvectorized.csv")

In [ ]:
df_lyrics.columns = ['lyrics_' + col for col in df_lyrics.columns]
df_lyrics

,lyrics_language,lyrics_joy,lyrics_sadness,lyrics_anger,lyrics_fear,lyrics_surprise,lyrics_disgust,lyrics_topic,lyrics_lyric_embed_0,lyrics_lyric_embed_1,...,lyrics_lyric_embed_758,lyrics_lyric_embed_759,lyrics_lyric_embed_760,lyrics_lyric_embed_761,lyrics_lyric_embed_762,lyrics_lyric_embed_763,lyrics_lyric_embed_764,lyrics_lyric_embed_765,lyrics_lyric_embed_766,lyrics_lyric_embed_767
0,47,0.010286,0.003357,0.003627,0.852823,0.073866,0.026022,-1,1.275576,-0.617951,...,-0.936317,0.580255,0.258699,0.300741,-0.029809,-0.133185,0.046484,-1.929412,1.467095,1.291935
1,10,0.010961,0.002502,0.007280,0.053901,0.026906,0.879299,111,-0.637413,-0.419503,...,-1.900526,0.209395,1.322138,-0.393067,0.053461,-1.230068,0.334036,0.992128,0.392356,-0.194402
2,10,0.077160,0.000983,0.001354,0.013347,0.005797,0.881736,101,0.180527,1.805151,...,0.468967,1.540109,-0.073639,-0.283662,-0.865780,-0.293890,-0.834001,0.158835,-1.304997,-0.474522
3,10,0.115452,0.008234,0.002304,0.574908,0.015430,0.274624,-1,-0.084946,-0.073967,...,0.021823,0.691457,0.135334,-0.847617,0.140568,-0.964377,0.368799,0.755176,-1.029618,0.032373
4,10,0.010929,0.017254,0.009780,0.007363,0.866119,0.039073,0,0.157111,-2.042170,...,1.365908,-1.529008,-1.493665,0.505125,-1.985446,1.811234,-2.134652,1.109510,1.298568,1.716432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,10,0.069033,0.001345,0.067869,0.063481,0.058327,0.030637,-1,1.179695,-0.323273,...,-0.183085,0.594693,0.304113,-0.057945,0.556560,-0.799339,0.340082,0.120239,-0.168247,0.813970
24980,10,0.201425,0.016703,0.026937,0.050839,0.192454,0.268406,2,-0.574946,0.057620,...,-1.264539,1.147598,-0.720407,0.258089,0.296109,-0.950428,1.755957,1.377265,-1.261943,0.095172
24981,11,0.297167,0.053284,0.121297,0.036948,0.393117,0.050207,3,-0.346286,-0.132091,...,1.550430,-0.792842,1.374321,-0.136573,0.039823,0.921098,0.670496,-1.454097,0.551432,-1.155509
24982,10,0.001478,0.000344,0.988243,0.005217,0.000673,0.001689,40,0.547461,0.215677,...,1.240856,-0.451447,-0.584143,-0.436976,0.096701,-0.777166,-0.338028,-0.147455,-0.784346,-0.035371


In [ ]:
df_names = pd.read_csv(r"C:\Users\rahul\Downloads\25knames.csv")

In [ ]:
df_names

,song_emb_0,song_emb_1,song_emb_2,song_emb_3,song_emb_4,song_emb_5,song_emb_6,song_emb_7,song_emb_8,song_emb_9,...,artist_emb_374,artist_emb_375,artist_emb_376,artist_emb_377,artist_emb_378,artist_emb_379,artist_emb_380,artist_emb_381,artist_emb_382,artist_emb_383
0,-0.389539,0.117288,-0.853330,-0.603562,-0.523745,1.591620,1.554765,-0.336767,-0.692799,0.546884,...,0.571664,0.417744,-0.278145,-0.263972,-0.004088,-0.294889,1.912058,1.341574,0.717148,1.687119
1,-1.416522,-0.613115,0.228263,-0.819381,-0.884185,-1.314516,3.006095,0.895698,-0.228470,0.338414,...,-0.419497,0.286736,-1.005038,-1.065450,-1.113877,-1.290094,-0.076251,-0.089463,-1.161524,2.488623
2,0.269100,-0.026498,-0.733111,1.016620,0.739821,-0.485052,0.159301,-0.722483,-0.767823,0.323434,...,-1.647708,-0.783465,0.443955,1.127369,0.676522,-0.249140,1.849778,-0.415092,0.781696,-0.204246
3,0.336696,-0.083116,-1.241477,-0.437091,1.234620,1.428652,0.489178,-1.371245,1.835731,1.504155,...,-1.077429,-0.515965,-0.313410,-1.096064,-0.651302,-1.412011,-2.317314,-0.888443,-0.113288,-0.214501
4,1.631066,0.029485,-0.764970,1.441404,-0.155687,0.554224,-0.526884,0.124584,-2.614832,2.262508,...,0.021903,0.913531,-0.404847,1.905599,-0.474565,-0.545335,-0.962313,0.279536,0.958003,-0.808991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,-0.484374,1.478709,1.546979,-0.022762,-0.719097,-0.906883,-0.566528,-0.926798,0.674475,-0.267511,...,-0.189350,1.436225,0.842273,0.992167,-0.251560,-0.131803,-0.391381,-0.464805,-1.012425,-1.271551
24980,-0.055678,-0.040190,-0.471570,-1.161510,-0.279047,0.474851,0.517613,-0.311240,0.404835,0.618024,...,-0.460759,-1.182036,-0.562780,-0.776367,-0.126819,-1.542312,1.683822,0.476537,0.564404,0.550906
24981,0.490019,-0.361707,-1.095892,-1.196160,0.210235,-0.607104,-0.439855,0.947622,1.055687,1.223532,...,-1.628479,-0.531543,2.048510,-0.630932,0.429318,-0.262628,-0.291085,-1.254308,-1.113364,-0.917095
24982,1.546311,0.845762,-0.839009,-2.216704,1.355024,0.545466,-0.777563,0.043220,2.233623,-1.228535,...,0.931303,0.353215,0.142943,0.035025,-0.216098,1.337191,0.904217,1.605344,1.063762,-0.459432


In [ ]:
df_names.columns = ['names_' + col for col in df_names.columns]
df_names

,names_song_emb_0,names_song_emb_1,names_song_emb_2,names_song_emb_3,names_song_emb_4,names_song_emb_5,names_song_emb_6,names_song_emb_7,names_song_emb_8,names_song_emb_9,...,names_artist_emb_374,names_artist_emb_375,names_artist_emb_376,names_artist_emb_377,names_artist_emb_378,names_artist_emb_379,names_artist_emb_380,names_artist_emb_381,names_artist_emb_382,names_artist_emb_383
0,-0.389539,0.117288,-0.853330,-0.603562,-0.523745,1.591620,1.554765,-0.336767,-0.692799,0.546884,...,0.571664,0.417744,-0.278145,-0.263972,-0.004088,-0.294889,1.912058,1.341574,0.717148,1.687119
1,-1.416522,-0.613115,0.228263,-0.819381,-0.884185,-1.314516,3.006095,0.895698,-0.228470,0.338414,...,-0.419497,0.286736,-1.005038,-1.065450,-1.113877,-1.290094,-0.076251,-0.089463,-1.161524,2.488623
2,0.269100,-0.026498,-0.733111,1.016620,0.739821,-0.485052,0.159301,-0.722483,-0.767823,0.323434,...,-1.647708,-0.783465,0.443955,1.127369,0.676522,-0.249140,1.849778,-0.415092,0.781696,-0.204246
3,0.336696,-0.083116,-1.241477,-0.437091,1.234620,1.428652,0.489178,-1.371245,1.835731,1.504155,...,-1.077429,-0.515965,-0.313410,-1.096064,-0.651302,-1.412011,-2.317314,-0.888443,-0.113288,-0.214501
4,1.631066,0.029485,-0.764970,1.441404,-0.155687,0.554224,-0.526884,0.124584,-2.614832,2.262508,...,0.021903,0.913531,-0.404847,1.905599,-0.474565,-0.545335,-0.962313,0.279536,0.958003,-0.808991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,-0.484374,1.478709,1.546979,-0.022762,-0.719097,-0.906883,-0.566528,-0.926798,0.674475,-0.267511,...,-0.189350,1.436225,0.842273,0.992167,-0.251560,-0.131803,-0.391381,-0.464805,-1.012425,-1.271551
24980,-0.055678,-0.040190,-0.471570,-1.161510,-0.279047,0.474851,0.517613,-0.311240,0.404835,0.618024,...,-0.460759,-1.182036,-0.562780,-0.776367,-0.126819,-1.542312,1.683822,0.476537,0.564404,0.550906
24981,0.490019,-0.361707,-1.095892,-1.196160,0.210235,-0.607104,-0.439855,0.947622,1.055687,1.223532,...,-1.628479,-0.531543,2.048510,-0.630932,0.429318,-0.262628,-0.291085,-1.254308,-1.113364,-0.917095
24982,1.546311,0.845762,-0.839009,-2.216704,1.355024,0.545466,-0.777563,0.043220,2.233623,-1.228535,...,0.931303,0.353215,0.142943,0.035025,-0.216098,1.337191,0.904217,1.605344,1.063762,-0.459432


In [ ]:
df_genres = pd.read_csv(r"C:\Users\rahul\Downloads\25kgenre.csv")

In [ ]:
df_genres.columns = ['genre_' + col for col in df_genres.columns]
df_genres

,genre_genre_cluster,genre_embedding_0,genre_embedding_1,genre_embedding_2,genre_embedding_3,genre_embedding_4,genre_embedding_5,genre_embedding_6,genre_embedding_7,genre_embedding_8,...,genre_embedding_758,genre_embedding_759,genre_embedding_760,genre_embedding_761,genre_embedding_762,genre_embedding_763,genre_embedding_764,genre_embedding_765,genre_embedding_766,genre_embedding_767
0,0,-0.559658,0.078598,-1.411476,0.317114,0.299750,-0.703917,0.048431,-0.295944,-0.167741,...,-0.253995,-0.919566,-0.836808,-0.288809,-2.088184,-0.690731,1.984316,-1.456049,1.126144,0.286425
1,2,0.071337,-0.671710,0.592621,-0.874156,1.078763,0.330534,-0.706646,-1.199230,-0.746443,...,-1.453121,0.966463,-1.027020,1.410087,0.975608,-0.409537,0.471199,-1.220358,-1.602603,-1.326023
2,7,-0.774547,-0.498703,0.941670,0.181002,0.585337,-1.279452,-1.029705,-1.523300,1.482859,...,0.536339,-1.079453,0.081693,-0.113450,0.216350,0.216963,-0.133482,0.044850,-1.203731,-0.698096
3,1,-0.087904,0.894951,0.745812,-0.587731,-0.461757,0.165065,-0.290217,1.155927,-1.266657,...,0.287984,0.427184,1.210508,-0.500069,-0.454545,-1.081239,-0.069704,-0.827256,-0.699773,0.740415
4,1,-0.087904,0.894951,0.745812,-0.587731,-0.461757,0.165065,-0.290217,1.155927,-1.266657,...,0.287984,0.427184,1.210508,-0.500069,-0.454545,-1.081239,-0.069704,-0.827256,-0.699773,0.740415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,8,-0.744596,-0.666107,-0.681363,1.568247,-0.866692,1.150038,0.512794,1.412163,0.473586,...,-1.241101,0.873541,-0.121854,0.227068,2.526002,1.126759,1.663650,0.176177,1.052313,0.999744
24980,0,0.459370,-0.235733,-1.617245,-0.461689,-0.097524,-1.866013,0.066015,0.782130,-0.234363,...,-0.367131,-0.935225,0.225364,-1.618141,-0.874078,-1.857174,0.064566,0.424953,0.133465,0.009819
24981,1,-0.087904,0.894951,0.745812,-0.587731,-0.461757,0.165065,-0.290217,1.155927,-1.266657,...,0.287984,0.427184,1.210508,-0.500069,-0.454545,-1.081239,-0.069704,-0.827256,-0.699773,0.740415
24982,0,1.356943,0.849672,-1.732249,0.913683,-2.288731,0.511937,-0.566667,1.147281,0.202975,...,-0.008767,2.899452,-0.267723,-0.306728,-0.150703,-0.003456,0.854176,-1.787693,0.120718,0.470449


25K work

In [ ]:
df = pd.read_csv(r"C:\Users\rahul\Downloads\combined_songs.csv")

In [ ]:
df

,song_title,song_id,track_popularity,track_duration,explicit,artist_name,artist_id,artist_genre,album_id,album_type,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Lyrics
0,Benim Babam (Bu Adam Benim Babam),7ijRJzpYlw1uyh0kJ6WPJp,35,332022,False,Fatih Kısaparmak,5JiRFs54W65l4k97s66Eef,"arabesk, turkish folk",4mAej5VFJ4hpQg0YoOK593,album,...,0.366,95.905,audio_features,7ijRJzpYlw1uyh0kJ6WPJp,spotify:track:7ijRJzpYlw1uyh0kJ6WPJp,https://api.spotify.com/v1/tracks/7ijRJzpYlw1u...,https://api.spotify.com/v1/audio-analysis/7ijR...,332022,4,2 ContributorsBu Adam Benim Babam LyricsBu Ada...
1,IDONTWANNASTAY,4y0NzIgyQJgNjZGDnmO7KD,39,159051,False,Kaphy,3wigq7ogMWIk2GhsT7BJC5,dark r&b,69NYnMmT4jr1dY94ni1V7Z,single,...,0.533,115.962,audio_features,4y0NzIgyQJgNjZGDnmO7KD,spotify:track:4y0NzIgyQJgNjZGDnmO7KD,https://api.spotify.com/v1/tracks/4y0NzIgyQJgN...,https://api.spotify.com/v1/audio-analysis/4y0N...,159052,4,1 ContributorIDONTWANNASTAY LyricsI'm tryna un...
2,Eyes Without A Face,0pUVeEgZuNyFzIMKp67RbS,76,299240,False,Billy Idol,7lzordPuZEXxwt9aoVZYmG,"album rock, classic rock, dance rock, hard roc...",2FZNWUmgRoP8uJZBaHJdfj,album,...,0.658,85.563,audio_features,0pUVeEgZuNyFzIMKp67RbS,spotify:track:0pUVeEgZuNyFzIMKp67RbS,https://api.spotify.com/v1/tracks/0pUVeEgZuNyF...,https://api.spotify.com/v1/audio-analysis/0pUV...,299240,4,55 ContributorsEyes Without a Face Lyrics\nI'm...
3,From a Logical Point of View,1rw7Ajqh0zjTpo29stGLu4,35,177983,False,Jensen Ackles,539FiNgLTBmfQzYMSTGhbi,No Genre,0UmSlRZUatcKfEAIZ7piYA,album,...,0.941,147.054,audio_features,1rw7Ajqh0zjTpo29stGLu4,spotify:track:1rw7Ajqh0zjTpo29stGLu4,https://api.spotify.com/v1/tracks/1rw7Ajqh0zjT...,https://api.spotify.com/v1/audio-analysis/1rw7...,177984,4,3 ContributorsFrom a Logical Point of View Lyr...
4,Gemendo Femininos,4uacHtbD37f4NuBhTD5pfc,2,130701,False,Djjazznobeat,0MhNJyXws2pdpTN1n9vBrW,No Genre,0G9LrjU7FgQhWJKivWOAcx,single,...,0.385,130.094,audio_features,4uacHtbD37f4NuBhTD5pfc,spotify:track:4uacHtbD37f4NuBhTD5pfc,https://api.spotify.com/v1/tracks/4uacHtbD37f4...,https://api.spotify.com/v1/audio-analysis/4uac...,130702,4,Likely instrumental or very low lyrics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,Revenge,0z3XcxcrqvWe36Dtgiqrh0,55,220208,False,Captainsparklez,6P9aim24wqJZ3SdoCWYwGT,"antiviral pop, pixel",2rpIKa3fNS6ijq0bu8v32S,single,...,0.761,119.939,audio_features,0z3XcxcrqvWe36Dtgiqrh0,spotify:track:0z3XcxcrqvWe36Dtgiqrh0,https://api.spotify.com/v1/tracks/0z3XcxcrqvWe...,https://api.spotify.com/v1/audio-analysis/0z3X...,220208,4,240 ContributorsTranslationsDeutschRevenge Lyr...
24980,Notti Bop,0IEpFiWXbmfpiiUHPlPkYC,61,225875,True,Kyle Richh,0hF6lbAjRsq4svrQUr5sgU,new york drill,7vkQS7TYsnbF0AopUZ7tjI,single,...,0.673,142.107,audio_features,0IEpFiWXbmfpiiUHPlPkYC,spotify:track:0IEpFiWXbmfpiiUHPlPkYC,https://api.spotify.com/v1/tracks/0IEpFiWXbmfp...,https://api.spotify.com/v1/audio-analysis/0IEp...,225875,4,"61 ContributorsNotti Bop Lyrics\nLike, 41K, y'..."
24981,Compás de Espera,10mwTAVbf2i7HoMV4xYdDO,20,185906,False,Maria Medina,0cuzLutbpu8OYFWwBh4THg,No Genre,7f1DiSnK7Rs1pktvnNJlc5,compilation,...,0.295,79.862,audio_features,10mwTAVbf2i7HoMV4xYdDO,spotify:track:10mwTAVbf2i7HoMV4xYdDO,https://api.spotify.com/v1/tracks/10mwTAVbf2i7...,https://api.spotify.com/v1/audio-analysis/10mw...,185907,4,2 ContributorsCompás de espera - a dueto con M...
24982,UEFA Europa League Anthem,7li2B5ovRTzE42gHvoxN8H,46,41160,False,UEFA,2dams9psI8ss1wZd2l62Xe,football,5JM6fxhx3LYvG1r1DtVlNR,single,...,0.704,175.533,audio_features,7li2B5ovRTzE42gHvoxN8H,spotify:track:7li2B5ovRTzE42gHvoxN8H,https://api.spotify.com/v1/tracks/7li2B5ovRTzE...,https://api.spotify.com/v1/audio-analysis/7li2...,41160,4,2 ContributorsChapter 1 LyricsCHAPTER ONE\nM. ...


In [ ]:
df_cleaned = df.drop(['id','song_id', 'album_id', 'track_duration','duration_ms', 'artist_name', 'artist_genre','track_href','uri','type','analysis_url','Lyrics','time_signature','artist_id','release_date'],axis=1)
df_cleaned

,song_title,track_popularity,explicit,album_type,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Benim Babam (Bu Adam Benim Babam),35,False,album,0.459,0.413,4,-14.976,0,0.2400,0.33800,0.00000,0.4290,0.366,95.905
1,IDONTWANNASTAY,39,False,single,0.776,0.765,7,-4.559,1,0.0658,0.01700,0.42200,0.2800,0.533,115.962
2,Eyes Without A Face,76,False,album,0.540,0.816,9,-6.197,1,0.0421,0.01770,0.00226,0.2760,0.658,85.563
3,From a Logical Point of View,35,False,album,0.801,0.343,7,-13.972,1,0.1310,0.81300,0.00000,0.1950,0.941,147.054
4,Gemendo Femininos,2,False,single,0.786,0.876,1,-15.881,1,0.1170,0.00472,0.96800,0.3220,0.385,130.094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,Revenge,55,False,single,0.748,0.830,7,-3.200,0,0.0527,0.22700,0.00000,0.0564,0.761,119.939
24980,Notti Bop,61,True,single,0.782,0.624,2,-10.807,1,0.4150,0.44000,0.00000,0.1190,0.673,142.107
24981,Compás de Espera,20,False,compilation,0.443,0.411,9,-9.637,1,0.0424,0.83200,0.00000,0.1180,0.295,79.862
24982,UEFA Europa League Anthem,46,False,single,0.304,0.659,1,-8.460,0,0.0493,0.08720,0.25600,0.3950,0.704,175.533


In [ ]:
# df_cleaned['Release Date'] = pd.to_datetime(df_cleaned['release_date'],format="%Y-%m-%d")
# datetime_columns = df_cleaned.select_dtypes(include=['datetime64']).columns
# for column in datetime_columns:
#     df_cleaned[f'{column}_year'] = df_cleaned[column].dt.year
#     df_cleaned[f'{column}_month'] = df_cleaned[column].dt.month
#     df_cleaned[f'{column}_day'] = df_cleaned[column].dt.day
#     df_cleaned[f'{column}_dayofweek'] = df_cleaned[column].dt.dayofweek
#     df_cleaned = df_cleaned.drop(column, axis=1)

In [ ]:
name = df['song_title']
# df_cleaned_no_name = df_cleaned.drop('track_name',axis=1)
# df_cleaned_no_name

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

df_cleaned['explicit'] = encoder.fit_transform(df_cleaned['explicit'])
df_cleaned['album_type'] = encoder.fit_transform(df_cleaned['album_type'])

In [ ]:
df_cleaned = df_cleaned.drop('song_title',axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df_cleaned)

In [ ]:
df_metadata = pd.DataFrame(scaled_df)
df_metadata.columns = df_cleaned.columns
df_metadata

,track_popularity,explicit,album_type,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.196998,-0.653331,-0.937865,-1.011994,-1.058443,-0.370076,-1.542451,-1.111983,0.945117,0.206303,-0.447376,1.469300,-0.562941,-0.928204
1,-0.029484,-0.653331,1.123416,0.817233,0.545446,0.464022,0.717147,0.899295,-0.464607,-0.892173,1.019194,0.529783,0.098774,-0.238551
2,1.520012,-0.653331,-0.937865,-0.544589,0.777827,1.020088,0.361841,0.899295,-0.656401,-0.889777,-0.439522,0.504561,0.594069,-1.283810
3,-0.196998,-0.653331,-0.937865,0.961494,-1.377398,0.464022,-1.324669,0.899295,0.063028,1.831772,-0.447376,-0.006183,1.715418,0.830537
4,-1.578981,-0.653331,1.123416,0.874938,1.051217,-1.204175,-1.738759,0.899295,-0.050268,-0.934195,2.916699,0.794613,-0.487656,0.247373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,0.640568,-0.653331,1.123416,0.655661,0.841618,0.464022,1.011934,-1.111983,-0.570619,-0.173544,-0.447376,-0.880123,1.002193,-0.101803
24980,0.891838,1.530617,1.123416,0.851856,-0.097021,-0.926142,-0.638135,0.899295,2.361315,0.555351,-0.447376,-0.485400,0.653505,0.660436
24981,-0.825172,-0.653331,0.092776,-1.104321,-1.067556,1.020088,-0.384345,0.899295,-0.653973,1.896791,-0.447376,-0.491705,-0.844268,-1.479837
24982,0.263664,-0.653331,1.123416,-1.906412,0.062456,-1.204175,-0.129036,-1.111983,-0.598134,-0.651945,0.442297,1.254913,0.776338,1.809778


In [ ]:
df_combined = pd.concat([df_metadata, df_lyrics, df_genres,df_names], axis=1)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Combine all DataFrames
df_combined = pd.concat([df_metadata, df_lyrics, df_genres,df_names], axis=1)

# Compute cosine similarity
cosine_sim_matrix = cosine_similarity(df_combined)

# Convert cosine similarity to a DataFrame for easier interpretation
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=name, columns=name)

# Print the cosine similarity DataFrame
# print(cosine_sim_df)

# Find top 10 pairs with highest cosine similarities
def find_top_similar_pairs(similarity_matrix, song_names, top_n=10):
    upper_triangle_indices = np.triu_indices_from(similarity_matrix, k=1)
    similarity_values = similarity_matrix[upper_triangle_indices]
    sorted_indices = np.argsort(similarity_values)[-top_n:][::-1]
    top_pairs = [(song_names[upper_triangle_indices[0][i]],
                  song_names[upper_triangle_indices[1][i]],
                  similarity_values[i]) for i in sorted_indices]
    return top_pairs

top_similar_pairs = find_top_similar_pairs(cosine_sim_matrix, name, top_n=10)

# Print the top 10 similar song pairs
print("Top 10 similar song pairs:")
for song1, song2, similarity in top_similar_pairs:
    print(f"'{song1}' and '{song2}' with similarity: {similarity}")


In [ ]:
df_combined

,track_popularity,explicit,album_type,danceability,energy,key,loudness,mode,speechiness,acousticness,...,names_artist_emb_374,names_artist_emb_375,names_artist_emb_376,names_artist_emb_377,names_artist_emb_378,names_artist_emb_379,names_artist_emb_380,names_artist_emb_381,names_artist_emb_382,names_artist_emb_383
0,-0.196998,-0.653331,-0.937865,-1.011994,-1.058443,-0.370076,-1.542451,-1.111983,0.945117,0.206303,...,0.571664,0.417744,-0.278145,-0.263972,-0.004088,-0.294889,1.912058,1.341574,0.717148,1.687119
1,-0.029484,-0.653331,1.123416,0.817233,0.545446,0.464022,0.717147,0.899295,-0.464607,-0.892173,...,-0.419497,0.286736,-1.005038,-1.065450,-1.113877,-1.290094,-0.076251,-0.089463,-1.161524,2.488623
2,1.520012,-0.653331,-0.937865,-0.544589,0.777827,1.020088,0.361841,0.899295,-0.656401,-0.889777,...,-1.647708,-0.783465,0.443955,1.127369,0.676522,-0.249140,1.849778,-0.415092,0.781696,-0.204246
3,-0.196998,-0.653331,-0.937865,0.961494,-1.377398,0.464022,-1.324669,0.899295,0.063028,1.831772,...,-1.077429,-0.515965,-0.313410,-1.096064,-0.651302,-1.412011,-2.317314,-0.888443,-0.113288,-0.214501
4,-1.578981,-0.653331,1.123416,0.874938,1.051217,-1.204175,-1.738759,0.899295,-0.050268,-0.934195,...,0.021903,0.913531,-0.404847,1.905599,-0.474565,-0.545335,-0.962313,0.279536,0.958003,-0.808991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,0.640568,-0.653331,1.123416,0.655661,0.841618,0.464022,1.011934,-1.111983,-0.570619,-0.173544,...,-0.189350,1.436225,0.842273,0.992167,-0.251560,-0.131803,-0.391381,-0.464805,-1.012425,-1.271551
24980,0.891838,1.530617,1.123416,0.851856,-0.097021,-0.926142,-0.638135,0.899295,2.361315,0.555351,...,-0.460759,-1.182036,-0.562780,-0.776367,-0.126819,-1.542312,1.683822,0.476537,0.564404,0.550906
24981,-0.825172,-0.653331,0.092776,-1.104321,-1.067556,1.020088,-0.384345,0.899295,-0.653973,1.896791,...,-1.628479,-0.531543,2.048510,-0.630932,0.429318,-0.262628,-0.291085,-1.254308,-1.113364,-0.917095
24982,0.263664,-0.653331,1.123416,-1.906412,0.062456,-1.204175,-0.129036,-1.111983,-0.598134,-0.651945,...,0.931303,0.353215,0.142943,0.035025,-0.216098,1.337191,0.904217,1.605344,1.063762,-0.459432


In [ ]:
df_vectorized_with_names = pd.concat([df['song_title'],df['artist_name'],df_combined],axis=1)

In [ ]:
df_vectorized_with_names

,song_title,artist_name,track_popularity,explicit,album_type,danceability,energy,key,loudness,mode,...,names_artist_emb_374,names_artist_emb_375,names_artist_emb_376,names_artist_emb_377,names_artist_emb_378,names_artist_emb_379,names_artist_emb_380,names_artist_emb_381,names_artist_emb_382,names_artist_emb_383
0,Benim Babam (Bu Adam Benim Babam),Fatih Kısaparmak,-0.196998,-0.653331,-0.937865,-1.011994,-1.058443,-0.370076,-1.542451,-1.111983,...,0.571664,0.417744,-0.278145,-0.263972,-0.004088,-0.294889,1.912058,1.341574,0.717148,1.687119
1,IDONTWANNASTAY,Kaphy,-0.029484,-0.653331,1.123416,0.817233,0.545446,0.464022,0.717147,0.899295,...,-0.419497,0.286736,-1.005038,-1.065450,-1.113877,-1.290094,-0.076251,-0.089463,-1.161524,2.488623
2,Eyes Without A Face,Billy Idol,1.520012,-0.653331,-0.937865,-0.544589,0.777827,1.020088,0.361841,0.899295,...,-1.647708,-0.783465,0.443955,1.127369,0.676522,-0.249140,1.849778,-0.415092,0.781696,-0.204246
3,From a Logical Point of View,Jensen Ackles,-0.196998,-0.653331,-0.937865,0.961494,-1.377398,0.464022,-1.324669,0.899295,...,-1.077429,-0.515965,-0.313410,-1.096064,-0.651302,-1.412011,-2.317314,-0.888443,-0.113288,-0.214501
4,Gemendo Femininos,Djjazznobeat,-1.578981,-0.653331,1.123416,0.874938,1.051217,-1.204175,-1.738759,0.899295,...,0.021903,0.913531,-0.404847,1.905599,-0.474565,-0.545335,-0.962313,0.279536,0.958003,-0.808991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24979,Revenge,Captainsparklez,0.640568,-0.653331,1.123416,0.655661,0.841618,0.464022,1.011934,-1.111983,...,-0.189350,1.436225,0.842273,0.992167,-0.251560,-0.131803,-0.391381,-0.464805,-1.012425,-1.271551
24980,Notti Bop,Kyle Richh,0.891838,1.530617,1.123416,0.851856,-0.097021,-0.926142,-0.638135,0.899295,...,-0.460759,-1.182036,-0.562780,-0.776367,-0.126819,-1.542312,1.683822,0.476537,0.564404,0.550906
24981,Compás de Espera,Maria Medina,-0.825172,-0.653331,0.092776,-1.104321,-1.067556,1.020088,-0.384345,0.899295,...,-1.628479,-0.531543,2.048510,-0.630932,0.429318,-0.262628,-0.291085,-1.254308,-1.113364,-0.917095
24982,UEFA Europa League Anthem,UEFA,0.263664,-0.653331,1.123416,-1.906412,0.062456,-1.204175,-0.129036,-1.111983,...,0.931303,0.353215,0.142943,0.035025,-0.216098,1.337191,0.904217,1.605344,1.063762,-0.459432


In [ ]:
import os
audio_dir = r"C:\Users\rahul\Downloads"
output_csv_final = os.path.join(audio_dir, '25kvectorizedFULLwithname.csv')
df_vectorized_with_names.to_csv(output_csv_final, index=False)
print(f"Final combined features saved to {output_csv_final}")

Final combined features saved to C:\Users\rahul\Downloads\25kvectorizedFULLwithname.csv


In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Assuming df_combined is your combined DataFrame with all relevant features
# Apply PCA to reduce dimensions
# pca = PCA(n_components=26)  # Retain 95% of variance
# reduced_features = pca.fit_transform(df_combined.values)

# Compute cosine similarity with reduced features
cosine_sim_matrix = cosine_similarity(df_combined.values)

# Convert to DataFrame for easier interpretation
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df_combined.index, columns=df_combined.index)

# Print the cosine similarity DataFrame
# print(cosine_sim_df)

# Function to find top similar pairs
def find_top_similar_pairs(sim_matrix, names, top_n=10):
    pairs = []
    for i in range(len(names)):
        for j in range(i + 1, len(names)):
            pairs.append((names[i], names[j], sim_matrix[i, j]))
    pairs.sort(key=lambda x: x[2], reverse=True)
    return pairs[:top_n]

# Find top 10 pairs with highest cosine similarities for reduced features
top_similar_pairs = find_top_similar_pairs(cosine_sim_matrix, df_combined.index, top_n=10)

# Print the top 10 similar song pairs with reduced features
print("Top 10 similar song pairs with reduced features:")
for song1, song2, similarity in top_similar_pairs:
    print(f"'{song1}' and '{song2}' with similarity: {similarity}")

Top 10 similar song pairs with reduced features:
'24' and '39' with similarity: 0.7477254892621895
'0' and '1' with similarity: 0.7058686520003467
'35' and '36' with similarity: 0.4944626603236725
'3' and '22' with similarity: 0.49220656688732345
'24' and '25' with similarity: 0.48117291558705483
'32' and '33' with similarity: 0.4795631367464626
'25' and '39' with similarity: 0.4403537090697684
'3' and '18' with similarity: 0.4243201425560552
'15' and '16' with similarity: 0.4149165684706969
'9' and '10' with similarity: 0.404177627808009


In [ ]:
# Define weights for each feature set
metadata_weight = 0.2
lyrics_weight = 0
# audio_weight = 0.1
genres_weight = 0.6
names_weight = 0.1

# Apply weights to each feature set
weighted_metadata = df_metadata.values * metadata_weight
weighted_lyrics = df_lyrics.values * lyrics_weight
# weighted_audio = df_audio.values * audio_weight
weighted_genres = df_genres.values * genres_weight
weighted_names  = df_names.values  * names_weight

# Combine weighted features
weighted_combined_features = np.hstack([
    weighted_metadata,
    weighted_lyrics,
    weighted_genres,
    weighted_names
])

# Compute cosine similarity with weighted features
weighted_cosine_sim_matrix = cosine_similarity(weighted_combined_features)

# Convert to DataFrame for easier interpretation
weighted_cosine_sim_df = pd.DataFrame(weighted_cosine_sim_matrix, index=name, columns=name)

# Print the weighted cosine similarity DataFrame
# print(weighted_cosine_sim_df)

# Find top 10 pairs with highest cosine similarities for weighted features
weighted_top_similar_pairs = find_top_similar_pairs(weighted_cosine_sim_matrix, name, top_n=20)

# Print the top 10 similar song pairs with weighted features
print("Top 10 similar song pairs with weighted features:")
for song1, song2, similarity in weighted_top_similar_pairs:
    print(f"'{song1}' and '{song2}' with similarity: {similarity}")


MemoryError: Unable to allocate 1.16 GiB for an array with shape (312087636,) and data type int32

In [ ]:

def get_cosine_metrics_by_index(index1, index2):
    if index1 >= len(name) or index2 >= len(name):
        raise ValueError(f"Indices {index1} and {index2} are out of range")

    song1 = name[index1]
    song2 = name[index2]

    sim = cosine_sim_df.iloc[index1, index2]
    # dist = cosine_dist_df.iloc[index1, index2]
    return song1, song2, sim

# Example usage
index1 = 1
index2 = 3400
song1, song2, similarity = get_cosine_metrics_by_index(index1, index2)
print(f"Cosine Similarity between '{song1}' and '{song2}': {similarity}")
# print(f"Cosine Distance between '{song1}' and '{song2}': {distance}")

Cosine Similarity between 'IDONTWANNASTAY' and 'Giant Steps - 2020 Remaster': 0.8559010624840173
